In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info[:10])
if gpu_info.find('failed') >= 0:
  print('failed, retry')
else:
  print(gpu_info)

In [ ]:
import os, math, sys, time, torch, pickle
from tqdm.notebook import tqdm_notebook as tq
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
%matplotlib inline

In [ ]:
base_path = ""
sys.path.append(base_path)

In [ ]:
from eval_utils.misc_utils import plot_figures
from eval_utils.generator import Generator
from eval_utils.dataset_utils import GenerateImages, NewDataset
from eval_utils.score_utils import ScoreModel
from eval_utils.calc_cifar10_stats import calc_cifar10_stats

In [ ]:
class options:
    def __init__(self):
        self.latent_dim = 100
        self.img_size = 32
        self.channels = 3
        self.cuda = True if torch.cuda.is_available() else False
opt = options()

In [ ]:
load_path = ""
g = Generator(opt)
if opt.cuda:
    g.cuda()
g.load_state_dict(torch.load(load_path + "generator_300.pth"))

In [ ]:
# store images
save_path = base_path + "saved_images/" + "sel_snr5_300/"
os.makedirs(save_path, exist_ok=True)

In [ ]:
gen_image_batches(g, opt=opt, save_path=save_path, n=10000)

In [ ]:
dset = NewDataset(root = save_path, transform=transforms.Compose([
        transforms.ToTensor(),
        ]))

score_model = ScoreModel(mode=2, stats_file=base_path + 'sel4_cifar10_stats.npz', cuda=True)
is_mean, is_std, fid = score_model.get_score_dataset(dset)
print(is_mean, is_std)
print('FID =', fid)

In [ ]:
dset = NewDataset(root = save_path, transform=transforms.Compose([
        transforms.ToTensor(),
        ]))

In [ ]:
dataloader = DataLoader(dset, batch_size=25)

In [ ]:
plot_figures(imgs)

In [ ]:
inception_score(dset, cuda=True, batch_size=32, resize=True, splits=10)

In [ ]:
# run this to calculate FID related stats on cifar10, 
# if needed provide a list 'select_classes' of required classes 
calc_cifar10_stats(base_path + "sel4_cifar10_stats", select_classes=[1,2,3,4])